## Log

- Bi-gram: 2.4716, 2.4755
- Single-head attention: 2.3899, 2.4041
- Multi-head attention, single layer: 2.0820, 2.1165
    - block_size = 8
    - hidden_size = 100
    - head_size = 20
    - head_num = 5

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
txt = open('shakespeare.txt', 'r').read()
len(txt)

5447119

In [3]:
txt[0:500]

"                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud"

In [4]:
chars = list(set(txt))
chars.sort()

ctoi = {c:i for i, c in enumerate(chars)}
itoc = {i:c for i, c in enumerate(chars)}
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !"&'(),-.0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}
84


In [5]:
i =  math.floor(0.9 * len(txt))
train_txt = txt[0:i]
valid_txt = txt[i+1:]

len(train_txt), len(valid_txt)

(4902407, 544711)

In [6]:
train_tkns = [ctoi[c] for c in train_txt]
valid_tkns = [ctoi[c] for c in valid_txt]

In [7]:
block_size = 8
batch_size = 32

def txt_to_token(t):
    return [ctoi[c] for c in t]
    
# (B, L)
def random_batch():
    xi = torch.randint(0, len(train_tkns)-block_size, (batch_size,))
    x = torch.tensor([train_tkns[i:i+block_size] for i in xi])
    y = torch.tensor([train_tkns[i+1:i+block_size+1] for i in xi])
    
    return x, y

x, y = random_batch()
x.shape

torch.Size([32, 8])

In [8]:
x[0]

tensor([ 1,  1,  1,  1,  1,  1,  1, 26])

In [9]:
def eval_split(split, model):
    tkn = train_tkns if split == "train" else valid_tkns
    tkn = torch.tensor(tkn)

    bsize = len(tkn) // block_size
    x = tkn[0:bsize*block_size]
    y = tkn[1:bsize*block_size+1]
    x = x.view(bsize, block_size) # (B, L)
    y = y.view(bsize, block_size) # (B, L)
    
    logits = model(x) # (B, L, C)
    B, L, C = logits.shape
    loss = F.cross_entropy(logits.view(B*L, C), y.view(B*L))
    
    return loss.item()

In [10]:
def sample(model):
    max_len = 500
    tks = [0]*block_size

    for i in range(max_len):
        ctx = torch.tensor(tks[i:i+block_size]) # (L)
        ctx = ctx.view(1, -1) # (B, L)

        logits = model(ctx) # (B, L, C)
        probs = F.softmax(logits, dim=2) # (B, L, C)
        probs = probs[0,-1,:] # (C), # the last in the sequence is the newly generated
        yi = torch.multinomial(probs, 1)
        tks.append(yi.item())

    chars = [itoc[t] for t in tks]
    return "".join(chars)

In [18]:
def fit(model):
    optim = torch.optim.Adam(model.parameters())
    
    for i in range(50000):
        optim.zero_grad()

        xb, yb = random_batch()
        logits = model(xb) # (B, L, C)

        B, L, C = logits.shape
        loss = F.cross_entropy(logits.view(B*L, C), yb.view(B*L))

        loss.backward()
        optim.step()

        if i % 5000 == 0:
            print(f"{loss.item():.4f}")

In [19]:
# Attension mechanism

head_size = 5
x = torch.rand(4, block_size, vocab_size) # (B, L, C)

key = nn.Linear(vocab_size, head_size)
query = nn.Linear(vocab_size, head_size)
value = nn.Linear(vocab_size, head_size)

k = key(x)   # (B, L, C)
q = query(x) # (B, L, C)
v = value(x) # (B, L, C)

q = q.permute(0, 2, 1) # (B, C, L)
w = k @ q # (B, L, L)

B, L, C = k.shape
mask = torch.tril(torch.ones(L, L))
mask = mask == 0
w = w.masked_fill(mask, -float('inf'))

prob = F.softmax(w, dim=2) # (B, L, L) TODO: not sure which dimension
a = prob @ v # (B, L, L) @ # (B, L, C)
a.shape # (B, L, C)

torch.Size([4, 8, 5])

In [20]:
class Head(nn.Module):    
    
    def __init__(self, in_size, head_size):
        super().__init__()
        
        self.head_size = head_size
        self.key = nn.Linear(in_size, head_size)
        self.query = nn.Linear(in_size, head_size)
        self.value = nn.Linear(in_size, head_size)
        
    # (B, L, C)  ->  (B, L, C')  
    def forward(self, x):
        k = self.key(x)   # (B, L, C')
        q = self.query(x) # (B, L, C')
        v = self.value(x) # (B, L, C')
        
        # TODO: other way than permute?
        q = q.permute(0, 2, 1) # (B, C', L)
        w = k @ q # (B, L, L)
        w /= self.head_size**0.5
        
        B, L, C = k.shape
        mask = torch.tril(torch.ones(L, L))
        mask = mask == 0
        w = w.masked_fill(mask, -float('inf'))

        prob = F.softmax(w, dim=2) # (B, L, L)
        a = prob @ v # (B, L, C')

        return a

In [21]:
class MultiHead(nn.Module):    
    
    def __init__(self, head_num, head_size, in_size, out_size):
        super().__init__()
        
        self.heads = [Head(in_size, head_size) for _ in range(head_num)]
        self.linear = nn.Linear(head_num*head_size, out_size, bias=False)

    # (B, L, C)  ->  (B, L, C')  
    def forward(self, x):
        hs = [h(x) for h in self.heads] 
        hs = torch.concat(hs, dim=2)  # (B, L, C)
        
        a = self.linear(hs)
        return a
        

x = torch.randn(2, 4, 9) # (B, L, C)
mh = MultiHead(5, 3, 9, 7)
mh(x).shape

torch.Size([2, 4, 7])

In [22]:
# return (L, C)
def pos_encoding(L, C):
    pos = torch.arange(0, L).view(-1, 1)
    d = torch.pow(10000, 2 * torch.arange(0, C) / C)
    u = pos * d

    pe[:,0::2] = torch.sin(u[:,0::2])
    pe[:,1::2] = torch.cos(u[:,0::2])
    return pe

In [23]:
hidden_size = 100
head_size = 20
head_num = 5

class Transformer(nn.Module):    
    
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.mheads = MultiHead(head_num, head_size, hidden_size, hidden_size)
        self.ffn1 = nn.Linear(hidden_size, hidden_size)
        self.ffn2 = nn.Linear(hidden_size, vocab_size)

    # (B, L) -> (B, L, C)
    def forward(self, x):
        y = self.embed(x) # (B, L, C)
        y = self.mheads(y) # (B, L, C)
        y = self.ffn1(y)
        y = torch.relu(y)
        y = self.ffn2(y)
        
        return y

In [24]:
tf_model = Transformer()

In [25]:
fit(tf_model)

4.4005
2.1424
2.2108
2.0654
2.2782
2.2726
2.0525
1.9083
2.0148
2.0887


In [ ]:
tr_loss = eval_split("train", tf_model)
va_loss = eval_split("valid", tf_model)

print(f"train: {tr_loss:.4f}")
print(f"valid: {va_loss:.4f}")

In [42]:
print(sample(tf_model))









    B >  h.Evganhilrds etl N ld  . c yer lin,  
n h,pr     t.vTr
 sxif
H U  y' t! EncTrr uvssr i.o d hr m I o oAtr 
 Tf DJi
 oawoei aee
hcl    yaa
ds  w   ,am edn oh ttohmlyhhhmaeethi
hteu e
 ha
nwt poerhho c ,etl asaf   hadb  h O cTIth ohgt ergen
oibus rta, s n H eerrmJa
ce Na, sdecawttuln  a agso rglpth rheerc.ecrsei   tl mmdlwtahI,ata
r os i  k Meeksllfm   aaw
uwyaidtwe !gyh ose alirl  l b ' oF-d!s  dt    tlhe heeel rdohete n mope B'wERfw p
ge  he  fta}dHsrAytae mhee d
shgr   Yfu [eoSed wmo a
